add volume thresholds and only trade when the underlying isn't going exponential against the spread positions Also only run a naive backtest

In [1]:
# --- Setup ---
import nest_asyncio
nest_asyncio.apply()

from ib_insync import *
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display

# --- Parameters ---
symbol1 = 'GOOG'
symbol2 = symbol1
lastTradeDateOrContractMonth1 = '20250725'
lastTradeDateOrContractMonth2 = '20250725'
strike1 = 180
strike2 = 160
right1 = 'P'
right2 = 'P'
barSize = '5 mins'
duration = '2 D'
window = 20
num_std = 2

# --- Connect to IBKR (async-safe for Jupyter) ---
ib = IB()
await ib.connectAsync('127.0.0.1', 7497, clientId=np.random.randint(1000, 9999))

# --- Define and qualify option contracts ---
option1 = Option(symbol1, lastTradeDateOrContractMonth1, strike1, right1, 'SMART', 'USD')
option2 = Option(symbol2, lastTradeDateOrContractMonth2, strike2, right2, 'SMART', 'USD')
await ib.qualifyContractsAsync(option1, option2)

# --- Subscribe to live bars ---
bars1 = ib.reqHistoricalData(
    option1, endDateTime='', durationStr=duration, barSizeSetting=barSize,
    whatToShow='TRADES', useRTH=True, formatDate=1, keepUpToDate=True
)
bars2 = ib.reqHistoricalData(
    option2, endDateTime='', durationStr=duration, barSizeSetting=barSize,
    whatToShow='TRADES', useRTH=True, formatDate=1, keepUpToDate=True
)

# --- Create persistent Plotly FigureWidget ---
fig = go.FigureWidget()
fig.add_scatter(name='Spread', line=dict(color='black'))
fig.add_scatter(name='BB Upper', line=dict(color='green', dash='dash'))
fig.add_scatter(name='BB Lower', line=dict(color='red', dash='dash'))
fig.add_scatter(name='BB Middle', line=dict(color='orange', dash='dot'))
fig.add_scatter(name='Buy Signal', mode='markers', marker=dict(symbol='triangle-up', color='green', size=12))
fig.add_scatter(name='Sell Signal', mode='markers', marker=dict(symbol='triangle-down', color='red', size=12))
fig.update_layout(
    title='Live Spread with Bollinger Bands and Signals',
    xaxis_title='Time',
    yaxis_title='Spread',
    template='plotly_white',
    width=1000,
    height=500
)
display(fig)

# --- Persistent state for delayed entry after exit ---
pending_entry = None

# --- Strategy + Plot Update Handler ---
def on_new_bar(bars, has_new_bar):
    global pending_entry
    if not has_new_bar:
        return

    # Convert to DataFrames
    df1 = util.df(bars1).set_index('date').rename(columns={'close': 'Close1'})
    df2 = util.df(bars2).set_index('date').rename(columns={'close': 'Close2'})

    # Align and calculate spread
    datafix = df1.join(df2, how='inner')
    datafix['spread'] = datafix['Close1'] - datafix['Close2']

    # Bollinger Bands
    datafix['bb_middle'] = datafix['spread'].rolling(window).mean()
    datafix['bb_std'] = datafix['spread'].rolling(window).std()
    datafix['bb_upper'] = datafix['bb_middle'] + num_std * datafix['bb_std']
    datafix['bb_lower'] = datafix['bb_middle'] - num_std * datafix['bb_std']

    # Signal logic
    datafix['prev_spread'] = datafix['spread'].shift(1)
    datafix['prev_bb_lower'] = datafix['bb_lower'].shift(1)
    datafix['prev_bb_upper'] = datafix['bb_upper'].shift(1)
    datafix['bb_signal'] = 'hold'
    buy_mask = (datafix['prev_spread'] > datafix['prev_bb_lower']) & (datafix['spread'] < datafix['bb_lower'])
    sell_mask = (datafix['prev_spread'] < datafix['prev_bb_upper']) & (datafix['spread'] > datafix['bb_upper'])
    datafix.loc[buy_mask, 'bb_signal'] = 'buy'
    datafix.loc[sell_mask, 'bb_signal'] = 'sell'

    # Get latest signal
    last_row = datafix.iloc[-1]
    signal = last_row['bb_signal']
    print(f"New bar at {last_row.name} | Spread: {last_row['spread']:.2f} | Signal: {signal}")

    # --- Trading logic (position-aware, exit before entry) ---
    positions = {p.contract.conId: p.position for p in ib.positions()}
    pos1 = positions.get(option1.conId, 0)
    pos2 = positions.get(option2.conId, 0)
    if pos1 == 1 and pos2 == -1:
        current_position = 'long'
    elif pos1 == -1 and pos2 == 1:
        current_position = 'short'
    else:
        current_position = 'flat'

    # Exit logic (and set pending entry)
    if current_position == 'long' and signal == 'sell':
        print("Exiting LONG spread...")
        ib.placeOrder(option1, MarketOrder('SELL', 1))
        ib.placeOrder(option2, MarketOrder('BUY', 1))
        pending_entry = 'short'
    elif current_position == 'short' and signal == 'buy':
        print("Exiting SHORT spread...")
        ib.placeOrder(option1, MarketOrder('BUY', 1))
        ib.placeOrder(option2, MarketOrder('SELL', 1))
        pending_entry = 'long'
    # Delayed entry: only enter after flat and pending_entry is set
    elif current_position == 'flat' and pending_entry is not None:
        if pending_entry == 'long':
            print("Delayed entry: Entering LONG spread...")
            ib.placeOrder(option1, MarketOrder('BUY', 1))
            ib.placeOrder(option2, MarketOrder('SELL', 1))
        elif pending_entry == 'short':
            print("Delayed entry: Entering SHORT spread...")
            ib.placeOrder(option1, MarketOrder('SELL', 1))
            ib.placeOrder(option2, MarketOrder('BUY', 1))
        pending_entry = None
    # If flat and no pending entry, act on new signals
    elif current_position == 'flat' and pending_entry is None:
        if signal == 'buy':
            print("Entering LONG spread...")
            ib.placeOrder(option1, MarketOrder('BUY', 1))
            ib.placeOrder(option2, MarketOrder('SELL', 1))
        elif signal == 'sell':
            print("Entering SHORT spread...")
            ib.placeOrder(option1, MarketOrder('SELL', 1))
            ib.placeOrder(option2, MarketOrder('BUY', 1))
        else:
            print("No action taken.")
    else:
        print("Holding or waiting for exit confirmation.")

    # --- Update plot ---
    plot_df = datafix.tail(50)
    buy_signals = plot_df[plot_df['bb_signal'] == 'buy']
    sell_signals = plot_df[plot_df['bb_signal'] == 'sell']
    with fig.batch_update():
        fig.data[0].x = plot_df.index
        fig.data[0].y = plot_df['spread']
        fig.data[1].x = plot_df.index
        fig.data[1].y = plot_df['bb_upper']
        fig.data[2].x = plot_df.index
        fig.data[2].y = plot_df['bb_lower']
        fig.data[3].x = plot_df.index
        fig.data[3].y = plot_df['bb_middle']
        fig.data[4].x = buy_signals.index
        fig.data[4].y = buy_signals['spread']
        fig.data[5].x = sell_signals.index
        fig.data[5].y = sell_signals['spread']

# --- Attach the update handler ---
bars1.updateEvent += on_new_bar
bars2.updateEvent += on_new_bar

print(" Live strategy, trading, and chart are running. Awaiting new bars...")


Error 200, reqId 3: No security definition has been found for the request, contract: Option(symbol='GOOG', lastTradeDateOrContractMonth='20250725', strike=180, right='P', multiplier='USD', exchange='SMART')
Error 200, reqId 4: No security definition has been found for the request, contract: Option(symbol='GOOG', lastTradeDateOrContractMonth='20250725', strike=160, right='P', multiplier='USD', exchange='SMART')
Unknown contract: Option(symbol='GOOG', lastTradeDateOrContractMonth='20250725', strike=180, right='P', multiplier='USD', exchange='SMART')
Unknown contract: Option(symbol='GOOG', lastTradeDateOrContractMonth='20250725', strike=160, right='P', multiplier='USD', exchange='SMART')
Error 200, reqId 5: No security definition has been found for the request, contract: Option(symbol='GOOG', lastTradeDateOrContractMonth='20250725', strike=180, right='P', multiplier='USD', exchange='SMART')
Error 200, reqId 6: No security definition has been found for the request, contract: Option(symbol=

FigureWidget({
    'data': [{'line': {'color': 'black'},
              'name': 'Spread',
              'type': 'scatter',
              'uid': '7c10d3c7-c1fa-45d8-9359-ed16685a1925'},
             {'line': {'color': 'green', 'dash': 'dash'},
              'name': 'BB Upper',
              'type': 'scatter',
              'uid': '036cd9c0-d0e2-4027-8ec6-4cb13f4cf244'},
             {'line': {'color': 'red', 'dash': 'dash'},
              'name': 'BB Lower',
              'type': 'scatter',
              'uid': '353f2767-afa8-4a7d-b156-2cb7c415b20b'},
             {'line': {'color': 'orange', 'dash': 'dot'},
              'name': 'BB Middle',
              'type': 'scatter',
              'uid': 'f2352934-3653-47d4-b157-8a72e55f3d06'},
             {'marker': {'color': 'green', 'size': 12, 'symbol': 'triangle-up'},
              'mode': 'markers',
              'name': 'Buy Signal',
              'type': 'scatter',
              'uid': 'aee4f228-a82e-4740-aa8c-1981e298e34e'},
           

 Live strategy, trading, and chart are running. Awaiting new bars...
